# **BCA Analysis Code 👩‍🔬🧪**

Hi! I originally made this summer 2024, but decided to make this code more simplified, include more instructions and notes, and add visualizations to improve understanding!

# 1. Set up

In [ ]:
#import the necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [ ]:
#read in the data

#make a table with the column 1 = Cell_Type (including standards) and column 2 = Raw_Absorbance
#download the raw data tab in excel as a CSV file
#then upload the csv using the tabs to the right
#click the three dots and click copy path
#paste the path into the ''
path_to_data = '.csv'
data = pd.read_csv(path_to_data, sep = ',')

In [ ]:
#take a look at the data
data

# 2. Normalize the Standards

In [ ]:
#subset the data to get the standards
#note in python the indexing starts at 0 and the upper boundary is exclusive
standards = data[0:]
standards #look at the standards

In [ ]:
#define a function that takes the raw absorbance of the 0 standard (iloc[0]) and subtracts that value from every other standard
#calling this function will return an updated, normalized standards dataframe
def normalize(dataframe, column):
  dataframe = dataframe.copy() #make a copy of the dataframe to avoid modifying the original data
  dataframe[column] = dataframe[column] - dataframe[column].iloc[0]
  return dataframe

In [ ]:
#implement the function to normalize the standards
#when implement this function call normalize(name of data frame, 'column name in data frame')
#save as a new data frame
normalized_standards = normalize(standards, 'Raw_Absorbance')
normalized_standards #look at the normalized standards

In [ ]:
#linear regression
#set x and y
x = normalized_standards['Cell_Type'].astype('float64')
y = normalized_standards['Raw_Absorbance'].astype('float64')

#linear regression
results = linregress(x,y)

#extract the slope, intercept, and R Squared values
slope_rounded, intercept_rounded, r_squared = round(results.slope, 7), round(results.intercept, 7), results.rvalue**2

#print
print('Slope:', slope_rounded)
print('Intercept:', intercept_rounded)
print('R^2:', r_squared)

In [ ]:
#visualize linear regression
x_line = np.linspace(x.min(), x.max(), 100)
y_line = slope_rounded * x_line + intercept_rounded

plt.figure(figsize=(9,6))
plt.grid(True, linestyle='--', alpha=0.5)
plt.scatter(x, y, color='deepskyblue', marker='*', s=100, alpha=0.8)
plt.plot(x_line, y_line, color='green', linestyle = 'dashed', linewidth=1)
equation_text = f'y = {slope_rounded}x + {intercept_rounded}\n$R^2$ = {r_squared}'
plt.text(0.18, 0.75, equation_text, transform=plt.gca().transAxes,
         fontsize=12, bbox=dict(facecolor='aliceblue', alpha=0.5))
plt.title('Visualization of Linear Regression', fontsize = 16)
plt.xlabel('Standards', fontsize = 14)
plt.ylabel('Raw Absorbance', fontsize = 14);


#3. How Much IP Buffer and 4X Dye to Add to Each Sample

In [ ]:
#subset the original data frame to get only the samples
BCA = data[:]
BCA #look at the data

In [ ]:
#ABS
#subtract the zero value from the samples and add a column with those values
BCA = BCA.copy()
zero_value = data['Raw_Absorbance'].loc[0]
BCA.loc[:, 'ABS'] = BCA['Raw_Absorbance'] - zero_value
BCA #look at dataframe

In [ ]:
#protein amount (ng/ul)
#calculate by using ABS values as x values in our equation
BCA['Protein_Amount'] = (BCA['ABS'] - intercept_rounded)/ slope_rounded
BCA

In [ ]:
#2x Dye
smallest_protein_amount = BCA['Protein_Amount'].min()
BCA['2X_Dye'] = (BCA['Protein_Amount']/ smallest_protein_amount)* #input total volume put in lysate tube
BCA

In [ ]:
#IP Buffer (ul) and 4X dye need to add
#Add column of total volume
BCA['IP_Buffer'] = (BCA['2X_Dye'] - 40).round(2)
BCA['4X_Dye'] = (BCA['2X_Dye'] / 3).round(2)
BCA['Total_Volume'] = BCA['IP_Buffer']+ BCA['4X_Dye'] + #input total volume in lysate tube
BCA


#4. How Many Blots Should I Run?

In [ ]:
#determine the maximum amount of protein if loaded 40 ul (maximum)
protein = (smallest_protein_amount/1000) * .75 #amount of protein ug/ul
protein_forty = protein * 40
print('ug of Protein per ul:', protein)
print('Protein amount in ug if loaded 40 ul:', protein_forty)

In [ ]:
#determine how much to load
protein_to_load = protein * #change number based on how much protein you want to load
print('If I load _ ul I will be loading', protein_to_load, 'ug of protein per well') #fill in number with amount loading

In [ ]:
#determine how many blots can do
smallest_volume = BCA['Total_Volume'].min()
blots = smallest_volume/ #change based on how much want to load
print('With loading _ ul I can run', blots, 'blots')